## +miRanda tool for target prediction of piRNA 

In [ ]:
import subprocess

def run_miranda_conda(input_fasta, input_sequence, energy, score, output_file):
    # Define the command using conda run to activate the environment and run miranda
    command = [
        'conda', 'run', '-n', 'mirenda', 'miranda',
        input_fasta,
        input_sequence,
        '-en', str(energy),
        '-sc', str(score),
        '-strict',
        '-out', output_file,
        '-quiet'
    ]
    
    # Run the command using subprocess and capture the output
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Miranda tool executed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.stderr}")  # No need to decode, e.stderr is already a string

# Example usage
input_fasta = '/home/engy/Documents/piRNA/our_tool/data/piRNAs_all.fasta'
input_sequence = '/home/engy/Documents/piRNA/miRANDA/gencode.v46.pc_transcripts.fa'
energy = -15
score = 120
output_file = '/home/engy/Documents/piRNA/our_tool/data/out2.txt'

run_miranda_conda(input_fasta, input_sequence, energy, score, output_file)

## Extracting the information from the output file of the miRanda tool

In [ ]:
import pandas as pd
from tqdm import tqdm

# Function to extract data from each block using string manipulation
def extract_data(block):
    # Initialize variables
    seq1 = seq2 = tot_score = tot_energy = max_score = max_energy = strand = len1 = len2 = None
    q_start = q_end = r_start = r_end = align_len = percentage1 = percentage2 = gene_name = gene_len = UTR5_start = UTR5_end = CDs_start = CDs_end = UTR3_start = UTR3_end= None

    lines = block.splitlines()

    # Extract data from the line that starts with ">>"
    for line in lines:
        if line.startswith('>>'):
            parts = line.split()
            seq1 = parts[0][2:]  # Remove the ">>" prefix
            seq2 = parts[1]
            gene_name = seq2.split("|")[5]
            gene_len = seq2.split("|")[6]
            
            try:
                UTR5_start = int(seq2.split("|")[7].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                UTR5_start = None  # Assign None or any default value if there's an issue
            
            try:
                UTR5_end = int(seq2.split("|")[7].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                UTR5_end = None
            
            try:
                CDs_start = int(seq2.split("|")[8].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                CDs_start = None
            
            try:
                CDs_end = int(seq2.split("|")[8].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                CDs_end = None
            
            try:
                UTR3_start = int(seq2.split("|")[9].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                UTR3_start = None
            
            try:
                UTR3_end = int(seq2.split("|")[9].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                UTR3_end = None

            tot_score = parts[2]
            tot_energy = parts[3]
            max_score = parts[4]
            max_energy = parts[5]
            strand = parts[6]
            len1 = parts[7]
            len2 = parts[8]
            break  
# Found the line, no need to continue searching

    # Extract Q, R, Align Len, and percentages from another line
    for line in lines:
        if 'Q:' in line and 'R:' in line:
            q_start = line[line.find('Q:')+2:line.find('to')].strip()
            q_end = line[line.find('to')+2:line.find('R:')].strip()
            r_start = line[line.find('R:')+2:line.find('to', line.find('R:'))].strip()
            r_end = line[line.find('to', line.find('R:'))+2:line.find('Align')].strip()
            align_len = line[line.find('Len')+5:line.find('Len')+7].strip()
            percentage1 = line[line.find('%')-5:line.find('%')].strip()
            percentage2 = line[line.rfind('(')+1:line.rfind('%')].strip()
            break

    return [seq1, seq2, tot_score, tot_energy, max_score, max_energy, strand, len1, len2, q_start , q_end, r_start , r_end, align_len, percentage1, percentage2, gene_name , gene_len,UTR5_start,UTR5_end, CDs_start,CDs_end,UTR3_start,UTR3_end]

 
# Read the text file containing multiple records
with open("out2.txt", "r") as file:
    blocks = file.read().split('Complete')

# Initialize an empty DataFrame with columns
columns = ['Seq1', 'Seq2', 'Tot Score', 'Tot Energy', 'Max Score', 'Max Energy', 'Strand', 'Len1', 'Len2', 'Q_start', 'Q_end' ,'R_start', "R_end" , 'Align Len', 'Percentage1', 'Percentage2', "gene_name" , "gene_len" , "UTR5_start", "UTR5_end", "CDs_start" , "CDs_end","UTR3_start","UTR3_end"]
df = pd.DataFrame(columns=columns)



# Process each block with a progress bar
for block in tqdm(blocks, desc="Processing Blocks", unit="block"):
    data = extract_data(block)
    if data and all(data[:2]):  # Ensure seq1 and seq2 are not  ";lkb nNone
        df = pd.concat([df, pd.DataFrame([data], columns=columns)], ignore_index=True)

# Save DataFrame to CSV fileout1.txt
df.to_csv("miranda_results_2.csv", index=False)

print("Data saved to miranda_results.csv")

**the same step with filtering**

In [ ]:
import pandas as pd
from tqdm import tqdm

# Function to extract data from each block using string manipulation
def extract_data(block):
    # Initialize variables
    seq1 = seq2 = tot_score = tot_energy = max_score = max_energy = strand = len1 = len2 = None
    q_start = q_end = r_start = r_end = align_len = percentage1 = percentage2 = gene_name = gene_len = UTR5_start = UTR5_end = CDs_start = CDs_end = UTR3_start = UTR3_end= None

    lines = block.splitlines()

    # Extract data from the line that starts with ">>"
    for line in lines:
        if line.startswith('>>'):
            parts = line.split()
            seq1 = parts[0][2:]  # Remove the ">>" prefix
            seq2 = parts[1]
            gene_name = seq2.split("|")[5]
            gene_len = int(seq2.split("|")[6])  # Convert gene length to integer for comparison
            try:
                UTR5_start = int(seq2.split("|")[7].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                UTR5_start = None  # Assign None or any default value if there's an issue
            
            try:
                UTR5_end = int(seq2.split("|")[7].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                UTR5_end = None
            
            try:
                CDs_start = int(seq2.split("|")[8].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                CDs_start = None
            
            try:
                CDs_end = int(seq2.split("|")[8].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                CDs_end = None
            
            try:
                UTR3_start = int(seq2.split("|")[9].split(":")[1].split("-")[0])
            except (IndexError, ValueError):
                UTR3_start = None
            
            try:
                UTR3_end = int(seq2.split("|")[9].split(":")[1].split("-")[1])
            except (IndexError, ValueError):
                UTR3_end = None

            tot_score = parts[2]
            tot_energy = parts[3]
            max_score = parts[4]
            max_energy = parts[5]
            strand = parts[6]
            len1 = parts[7]
            len2 = parts[8]
            break  # Found the line, no need to continue searching

    # Extract Q, R, Align Len, and percentages from another line
    for line in lines:
        if 'Q:' in line and 'R:' in line:
            q_start = line[line.find('Q:')+2:line.find('to')].strip()
            q_end = line[line.find('to')+2:line.find('R:')].strip()
            r_start = line[line.find('R:')+2:line.find('to', line.find('R:'))].strip()
            r_end = line[line.find('to', line.find('R:'))+2:line.find('Align')].strip()
            align_len = line[line.find('Len')+5:line.find('Len')+7].strip()
            percentage1 = line[line.find('%')-5:line.find('%')].strip()
            percentage2 = line[line.rfind('(')+1:line.rfind('%')].strip()
            break

    return [seq1, seq2, tot_score, tot_energy, max_score, max_energy, strand, len1, len2, q_start , q_end, r_start , r_end, align_len, percentage1, percentage2, gene_name , gene_len, UTR5_start, UTR5_end, CDs_start, CDs_end, UTR3_start, UTR3_end]


# Read the text file containing multiple records
with open("out2.txt", "r") as file:
    blocks = file.read().split('Complete')

# Initialize an empty DataFrame with columns
columns = ['Seq1', 'Seq2', 'Tot Score', 'Tot Energy', 'Max Score', 'Max Energy', 'Strand', 'Len1', 'Len2', 'Q_start', 'Q_end', 'R_start', 'R_end', 'Align Len', 'Percentage1', 'Percentage2', 'gene_name', 'gene_len', 'UTR5_start', 'UTR5_end', 'CDs_start', 'CDs_end', 'UTR3_start', 'UTR3_end']
df = pd.DataFrame(columns=columns)

# Dictionary to store the longest gene length for each gene
gene_max_len = {}

# Process each block with a progress bar
for block in tqdm(blocks, desc="Processing Blocks", unit="block"):
    data = extract_data(block)
    
    if data and all(data[:2]):  # Ensure seq1 and seq2 are not None
        gene_name = data[15]
        gene_len = data[16]
        
        # If the gene has been seen before, check if this one is longer
        if gene_name in gene_max_len:
            if gene_len > gene_max_len[gene_name]:
                # Update with the new longer gene
                gene_max_len[gene_name] = gene_len
                # Replace previous entry in the DataFrame with this one
                df = df[df['gene_name'] != gene_name]  # Remove previous entry
                df = pd.concat([df, pd.DataFrame([data], columns=columns)], ignore_index=True)
        else:
            # New gene, add to both dictionary and DataFrame
            gene_max_len[gene_name] = gene_len
            df = pd.concat([df, pd.DataFrame([data], columns=columns)], ignore_index=True)

# Save DataFrame to CSV file
df.to_csv("miranda_results.csv", index=False)

print("Data saved to miranda_results.csv")
